In [17]:
### Change between "cat" and "dog" in the penultimate cell to evaluate both models precision and recall.

In [1]:
import torch
import numpy as np

from torchvision import models
import torch.nn as nn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.filterwarnings("ignore")


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
cat_train_path = "assets/transformed data/train_data_cat.pkl"
cat_test_path = "assets/transformed data/test_data_cat.pkl"
dog_train_path = "assets/transformed data/train_data_dog.pkl"
dog_test_path = "assets/transformed data/test_data_dog.pkl"

In [5]:
# --------- MODEL ---------
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1280, 128),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.fc(x)
        return x

In [7]:
# --------- EMBEDDING GENERATION ---------
def save_gallery_embeddings(model, dataset_path, animal):
    model.eval()
    model.to(DEVICE)
    embeddings = []
    
    with open(dataset_path, "rb") as f:
        dataset = pickle.load(f)

    data_label_1 = [item[0] for item in dataset if item[1] == 1]

    for img_tensor_norm in data_label_1:
        img_tensor = img_tensor_norm.unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            emb = model(img_tensor).cpu().numpy().flatten()
            embeddings.append(emb)
        
    with open(f"assets/embedding gallery/my_{animal}_gallery.pkl", "wb") as f:
        pickle.dump(np.array(embeddings), f)
    print("Gallery embeddings saved.")

In [9]:
# --------- VERIFICATION ---------
def verify_folder(model, dataset_path, gallery_path, threshold=0.75):
    
    with open(gallery_path, "rb") as f:
        gallery = pickle.load(f)

    with open(dataset_path, "rb") as f:
        dataset = pickle.load(f)

    model.eval()
    model.to(DEVICE)
    test_images = [item[0] for item in dataset]
    true_labels = [item[1] for item in dataset]
    predictions = []

    match_count = 0
    total_count = 0

    for img in test_images:
        tensor = img.unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            emb = model(tensor).cpu().numpy().flatten()
        score = np.mean(cosine_similarity([emb], gallery))
        if score >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)
        
    # Calculate precision and recall
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions)
    
    # print(f"Precision: {precision:.2f}")
    # print(f"Recall: {recall:.2f}")

    return precision, recall


##### Change between cat and dog to verify the models with test datasets

In [11]:
animal = "dog"


if animal == "cat":
    train_path = cat_train_path
    test_path = cat_test_path

elif animal == "dog":
    train_path = dog_train_path
    test_path = dog_test_path

model = EmbeddingNet()

# Load the saved weights
model.load_state_dict(torch.load(f"assets/trained models/{animal}_model.pt"))


save_gallery_embeddings(model, train_path, animal)

Gallery embeddings saved.


In [13]:
verify_folder(model, test_path, f"assets/embedding gallery/my_{animal}_gallery.pkl", threshold=0.85)

(1.0, 1.0)